In [ ]:
// Import max-plus algebra functions from our library
import { 
  oplus, otimes, odot, transpose, trace, star, plus, powOtimes,
  EPS, INF, Matrix, Vector,
  createEpsilonMatrix, createIdentityMatrix, reset as resetMatrix
} from './max-plus';

import * as tslab from 'tslab';

### Delay
Note: the actual delay tested might not be 100 ms.

<img src="../../img/Delay100ms.svg" alt="drawing" width="500"/>

In [ ]:
// System Max-Plus Encoding

// Create matrices for system
let x: Matrix = [[EPS, EPS]];

// Indices for system
const a = 0;
const b = 1;

console.log("System setup complete");


In [ ]:
// Logical time
let t: number = EPS;
let tBar: Matrix = [[t, t]];

// Timer
const offset = 0;
const period = 1; // A period of 0.15 (150ms) is too small, in which case the barrier sync constraints will dominate.
let timerValue: number = offset;

// Delay
const delay = .5; // A delay < 200 ms should have no effect on the firing time because of barrier sync.

// Execution times
let e: Matrix = [[0.1, 0.1]];

// Identity matrix
const Identity: Matrix = [
  [0, EPS],
  [EPS, 0],
];

// Dependency matrix
let Gamma: Matrix;
let GammaStar: Matrix;

// Barrier synchronization matrix
let B: Matrix;


In [ ]:
function updateLogicalTime(kPrime: number): void {
  if (kPrime == 0) t = timerValue; // initial offset
  // If kPrime is odd, add delay on top of t.
  else if (kPrime % 2 == 1) t += delay;
  // If kPrime is even, increment timerValue.
  else {
    timerValue += period;
    t = timerValue; 
  }
  // Update tBar.
  tBar = [[t, t]];
}

function updateApparentLatency(kPrime: number): void {
  // If k' is odd (k is even), both reactions fire.
  if (kPrime % 2 == 1) e = [[0.1, 0.1]];
  // If k' is even (k is odd), only reaction A fires.
  else e = [[0.1, 0]];

  // Update dependency matrix
  Gamma = [
    [0, EPS],
    [e[0][a], 0],
  ];
  
  // GammaStar is normally calculated like this (Baccelli 1.22):
  // A^* = e ⊕ A ⊕ ···⊕ A^n ⊕ A^(n+1) ⊕ ···
  //
  // But here we take a shortcut by using Theorem 3.20:
  // A^* = e ⊕ A ⊕ ··· ⊕ A^(n−1)
  // where n is the dimension of matrix A.
  // 
  // To compute GammaStar this way,
  // the sufficient condition must be satisfied,
  // which is Gamma should not have any circuit with
  // a positive weight.
  GammaStar = oplus(Identity, Gamma);

  // Update barrier synchronization matrix
  B = [
    [e[0][a], e[0][b]],
    [e[0][a], e[0][b]],
  ];

  console.log("k'=", kPrime, " k=", kPrime-1);
  console.log("e:", e);
  console.log("Gamma:", Gamma);
  console.log("GammaStar:", GammaStar);
  console.log("B:", B);
}

In [ ]:
// Evolution equations for system in factored form:
// x' = (GammaStar ⊗ B) ⊗ x ⊕ (GammaStar ⊗ tBar)
function step(x: Matrix, tBar: Matrix): Matrix {
  const xT: Matrix = transpose(x);
  const tT: Matrix = transpose(tBar);

  // (GammaStar ⊗ B) ⊗ x
  const GB = otimes(GammaStar, B);
  console.log("GB:", GB);
  const GBx = otimes(GB, xT);
  console.log("GBx:", GBx);

  // GammaStar ⊗ tBar
  const Gt = otimes(GammaStar, tT);
  console.log("Gt:", Gt);

  // Combine and transpose back to row vector
  const result = oplus(GBx, Gt);
  return transpose(result);
}

// Reporting function (updated to work with matrices)
function reportStats(k: number, x: Matrix, t: number): void {
  const lagK = x[0].map(xi => xi - t); // lag

  console.log(`Tag index k=${k}`);
  console.log(`t(k)= ${t}`);
  console.log(`x(k) = earliest possible firing times = [${x[0].join(', ')}]`);
  console.log(`lag(k) = [${lagK.join(', ')}]`);
}  

console.log("System functions defined");


In [ ]:
// System simulation setup
// Initial logical time (scalar then blasted)
t = 0;
tBar = [[t, t]];

// Reset x to eps using the library function
x = resetMatrix(x);

// Iteration count
let k = 0;

console.log("System simulation ready");


### Execute the following repeatly.

In [ ]:
let xArr: Matrix[] = [];
let tArr: number[] = [];
let eArr: number[][] = [];

// Unroll execution for 10 times.
for (let k = 0; k < 10; k++) {
  eArr.push(e[0]); // Store e(k)

  // Update logical time and apparent latency.
  updateLogicalTime(k);
  updateApparentLatency(k);

  // Step the system from k to k+1
  x = step(x, tBar);
  xArr.push(x); // Store x(k')
  tArr.push(t); // Store t(k')
  console.log("x(", k, ") =", x);

  // Statistics
  // reportStats(k, x, t);
}


## HTML Timeline Export

Let's create a function to generate and save an HTML timeline file.


In [ ]:
// Extract timeline data from xArr (generic for any number of reactions)
function extractTimelineData(xArr: Matrix[], tArr: number[] = []) {
  const reactions: { [key: string]: Array<{tagIndex: number, time: number, logicalTime: number, lag: number, execTime: number}> } = {};
  
  if (xArr.length === 0) return { reactionA: [], reactionB: [] };
  
  // Initialize reaction arrays based on matrix width
  const numReactions = xArr[0][0].length;
  const reactionNames = Array.from({ length: numReactions }, (_, i) => String.fromCharCode(65 + i)); // A, B, C, D, ...
  
  reactionNames.forEach(name => {
    reactions[name] = [];
  });
  
  xArr.forEach((matrix, k) => {
    const logicalTime = tArr.length > k ? tArr[k] : 0; // Get logical time for this iteration
    matrix[0].forEach((time, reactionIndex) => {
      if (time !== EPS && time !== INF) {
        const reactionName = reactionNames[reactionIndex];
        const lag = time - logicalTime; // Calculate lag as x - t
        const execTime = eArr.length > k ? eArr[k][reactionIndex] : 0; // Get logical time for this iteration
        reactions[reactionName].push({ 
          tagIndex: k, 
          time: time, 
          logicalTime: logicalTime,
          lag: lag,
          execTime: execTime,
        });
      }
    });
  });
  
  // For backward compatibility, still return reactionA and reactionB
  return { 
    reactionA: reactions['A'] || [], 
    reactionB: reactions['B'] || [],
    ...reactions
  };
}

console.log("Timeline data extraction function defined");


### Generate HTML Timeline

Create a comprehensive HTML timeline with Vis.js integration.


In [ ]:
// Generate comprehensive HTML timeline with Vis.js (generic for any number of reactions)
function generateHTMLTimeline(xArr: Matrix[], tArr: number[] = []) {
  const reactionData = extractTimelineData(xArr, tArr);
  
  // Create a generic color palette for reactions
  const colors = ['#3498db', '#e74c3c', '#2ecc71', '#f39c12', '#9b59b6', '#1abc9c', '#e67e22', '#34495e'];
  
  // Create Vis.js timeline items and groups dynamically
  const items: any[] = [];
  const groups: any[] = [];
  const allReactions: Array<{name: string, events: Array<{tagIndex: number, time: number, logicalTime: number, lag: number, execTime: number}>, color: string}> = [];
  
  // Build reaction list dynamically
  Object.keys(reactionData).forEach((key, index) => {
    // Skip the backward compatibility keys and only use the actual reaction keys
    if (key.length === 1 && reactionData[key].length > 0) {
      allReactions.push({
        name: key,
        events: reactionData[key],
        color: colors[index % colors.length]
      });
    }
  });
  
  // Create groups
  allReactions.forEach((reaction) => {
    groups.push({
      id: reaction.name,
      content: `Reaction ${reaction.name}`,
      style: `background-color: white; color: #2c3e50; border-left: 4px solid ${reaction.color};`
    });
  });
  
  // Create items
  allReactions.forEach((reaction) => {
    reaction.events.forEach(event => {
      // Create multi-line content with k + index, logical time, and lag
      const content = `k = ${event.tagIndex}<br/>x = ${event.time.toFixed(3)}<br/>t = ${event.logicalTime.toFixed(3)}<br/>lag = ${event.lag.toFixed(3)}<br/>e = ${event.execTime.toFixed(3)}`;
      
      items.push({
        id: `${reaction.name}_${event.tagIndex}`,
        content: content,
        start: event.time * 1000, // Use milliseconds from epoch as relative time
        group: reaction.name,
        title: `Tag ${event.tagIndex}: Reaction ${reaction.name} fires at ${event.time.toFixed(3)}s (logical time: ${event.logicalTime.toFixed(3)}s, lag: ${event.lag.toFixed(3)}s)`,
        style: `background-color: ${reaction.color}; border-color: ${reaction.color}; color: white;`
      });
    });
  });
  
  // Calculate statistics
  const totalEvents = items.length;
  
  // Calculate average delays between consecutive reactions if they exist
  let avgDelay = 0;
  if (allReactions.length >= 2 && xArr.length > 0) {
    const delays: number[] = [];
    xArr.forEach((matrix, k) => {
      for (let i = 0; i < matrix[0].length - 1; i++) {
        const time1 = matrix[0][i];
        const time2 = matrix[0][i + 1];
        if (time1 !== EPS && time1 !== INF && time2 !== EPS && time2 !== INF) {
          delays.push(Math.abs(time2 - time1));
        }
      }
    });
    avgDelay = delays.length > 0 ? delays.reduce((sum, d) => sum + d, 0) / delays.length : 0;
  }
  
  const htmlContent = `<!DOCTYPE HTML>
<html>
<head>
  <title>CAL Timeline</title>
  <style type="text/css">
    body, html {
      font-family: sans-serif;
    }
  </style>
  <script src="https://unpkg.com/vis-timeline@latest/standalone/umd/vis-timeline-graph2d.min.js"></script>
  <link href="https://unpkg.com/vis-timeline@latest/styles/vis-timeline-graph2d.min.css" rel="stylesheet" type="text/css" />
</head>
<body>
<div id="visualization"></div>

<script type="text/javascript">
  // DOM element where the Timeline will be attached
  var container = document.getElementById('visualization');

  // Create a DataSet (allows two way data-binding)
  var items = new vis.DataSet(${JSON.stringify(items)});
  var groups = new vis.DataSet(${JSON.stringify(groups)});

  // Configuration for the Timeline
  var options = {
    format: {
      minorLabels: function(date, scale, step) {
        const seconds = (typeof date === 'number' ? date : new Date(date).getTime()) / 1000;
        return seconds.toFixed(1) + 's';
      },
      majorLabels: function(date, scale, step) {
        const seconds = (typeof date === 'number' ? date : new Date(date).getTime()) / 1000;
        return seconds.toFixed(2) + 's';
      }
    }
  };

  // Create a Timeline
  var timeline = new vis.Timeline(container, items, groups, options);
</script>
</body>
</html>`;

  return htmlContent;
}

console.log("HTML timeline generation function defined");


In [ ]:
//// Trying to display HTML within the notebook. Does not work.
// const htmlContent = generateHTMLTimeline(xArr, tArr);
// tslab.display.html(htmlContent);

### Write HTML Timeline to File

Function to actually write the HTML timeline to a file on disk.


In [ ]:
// Function to write HTML timeline to file (Node.js only)
function writeTimelineToFile(xArr: Matrix[], tArr: number[] = [], filename: string = 'reaction-timeline.html') {
  const htmlContent = generateHTMLTimeline(xArr, tArr);
  
  try {
    // Check if we're in a Node.js environment
    if (typeof require !== 'undefined') {
      const fs = require('fs');
      const path = require('path');
      
      // Write to current directory
      const fullPath = path.resolve(filename);
      fs.writeFileSync(fullPath, htmlContent, 'utf8');
      
      console.log(`✅ Timeline saved to: ${fullPath}`);
      console.log(`🌐 Open the file in your browser to view the interactive timeline`);
      
      return fullPath;
    } else {
      // Not in Node.js - just output the HTML content
      console.log(`⚠️  File writing not available in this environment`);
      console.log(`📋 Copy the HTML content below and save as '${filename}':`);
      console.log('='.repeat(60));
      console.log(htmlContent);
      console.log('='.repeat(60));
      return null;
    }
  } catch (error) {
    console.error('❌ Error writing timeline file:', error);
    console.log('\n📋 HTML Content (copy and save manually):');
    console.log('='.repeat(50));
    console.log(htmlContent);
    console.log('='.repeat(50));
    return null;
  }
}

// Alternative function to just get the HTML content
function getTimelineHTML(xArr: Matrix[], tArr: number[] = []): string {
  return generateHTMLTimeline(xArr, tArr);
}

// Create and save the timeline
console.log("🚀 Creating enhanced timeline with logical times and lag...");
const result = writeTimelineToFile(xArr, tArr, 'reaction-timeline.html');

// Also provide the HTML content for manual saving
console.log("\n💡 Alternative: Get HTML content directly:");
console.log("const htmlContent = getTimelineHTML(xArr, tArr);");
